In [11]:
from model import AwaleNetwork
import jax
from awale.env import AwaleJAX
import equinox as eqx
from jax import numpy as jnp
from jax import random



In [2]:
env = AwaleJAX( )

In [3]:
key = jax.random.PRNGKey(0)
state = env.reset(key)

In [21]:
class DQNAgent :
    def __init__(self , key) :
        self.memory = [ ]
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = AwaleNetwork(key)

    def remember(self , state , action , reward , next_state , done) :
        memory_size = 2000
        experience = (state , action , reward , next_state , done)
        self.memory.append(experience)
        if len(self.memory) > memory_size :
            self.memory.pop(0)

    def act(self , key , state , action_space) :
        if random.uniform(key) <= self.epsilon :
            return random.choice(key , action_space)
        q_values = self.model.predict(state)
        return jnp.argmax(q_values[ 0 ])

    def replay(self , batch_size , key) :
        # Generate random indices using JAX
        indices = jax.random.permutation(key , len(self.memory))[ :batch_size ]
        minibatch = [ self.memory[ int(idx) ] for idx in indices ]
        for state , action , reward , next_state , done in minibatch :
            target = reward
            if not done :
                print(self.model(next_state.board , next_state.action_space , next_state.score))
                target = reward + self.gamma * jnp.amax(self.model(next_state.board , next_state.action_space , next_state.score))
                print(target)
            target_f = self.model.predict(state)
            target_f[ 0 ][ action ] = target
            self.model.fit(state , target_f , epochs=1 , verbose=0)
        if self.epsilon > self.epsilon_min :
            self.epsilon *= self.epsilon_decay

In [24]:
jax.random.choice(key , jnp.array([ 1 , 2 , 3 ]))

Array(1, dtype=int32)

In [22]:
# Initialize the DQN agent
agent = DQNAgent(key)

In [15]:
agent.model

AwaleNetwork(
  state_encoder=[
    Linear(
      weight=f32[64,14],
      bias=f32[64],
      in_features=14,
      out_features=64,
      use_bias=True
    ),
    Linear(
      weight=f32[128,64],
      bias=f32[128],
      in_features=64,
      out_features=128,
      use_bias=True
    ),
    <wrapped function relu>,
    Linear(
      weight=f32[64,128],
      bias=f32[64],
      in_features=128,
      out_features=64,
      use_bias=True
    ),
    <wrapped function relu>
  ],
  action_embedding=Embedding(
    num_embeddings=12,
    embedding_size=32,
    weight=f32[12,32]
  ),
  combine=[
    Linear(
      weight=f32[64,96],
      bias=f32[64],
      in_features=96,
      out_features=64,
      use_bias=True
    ),
    <wrapped function relu>,
    Linear(
      weight=f32[32,64],
      bias=f32[32],
      in_features=64,
      out_features=32,
      use_bias=True
    ),
    <wrapped function relu>,
    Linear(
      weight=f32[1,32],
      bias=f32[1],
      in_features=32,
      

In [6]:
# Training loop
batch_size = 32
num_episodes = 1000

In [23]:
for episode in range(num_episodes) :
    # random initial state
    key = jax.random.PRNGKey(episode)
    state = env.reset(key)
    for time in range(500) :
        # Choose action
        key = jax.random.PRNGKey(time)
        action = agent.act(key , state.board , state.action_space)
        # Take action
        next_state , reward , done = env.step(state , action)
        # Remember the experience
        agent.remember(state , action , reward , next_state , done)
        # Update the state
        state = next_state
        # Check if episode is finished
        if done :
            break
        # Train the agent
        print(len(agent.memory))
        if len(agent.memory) > batch_size :
            agent.replay(batch_size , key)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
[0.11819653 0.13046151 0.03171215 0.13640115 0.03101879]
-4.370419


AttributeError: 'AwaleNetwork' object has no attribute 'predict'